In [1]:
# @title Environment Setup
import os
import sys
if 'MPLBACKEND' in os.environ:
    del os.environ['MPLBACKEND']
    print("MPLBACKEND environment variable cleared.")

# 2. Clone the repository
!rm -rf FontDiffusion
!git clone https://github.com/dzungphieuluuky/FontDiffusion.git

MPLBACKEND environment variable cleared.
Cloning into 'FontDiffusion'...
remote: Enumerating objects: 20086, done.
remote: Counting objects: 100% (4898/4898), done.
remote: Compressing objects: 100% (4877/4877), done.
remote: Total 20086 (delta 49), reused 4858 (delta 21), pack-reused 15188 (from 3)
Receiving objects: 100% (20086/20086), 277.16 MiB | 10.54 MiB/s, done.
Resolving deltas: 100% (612/612), done.
Updating files: 100% (136/136), done.


In [2]:
import os
from IPython import get_ipython
from typing import Optional

def configure_environment_paths():
    """Detect environment and configure paths"""
    try:
        if "google.colab" in str(get_ipython()):
            print("✅ Environment: Google Colab")
            base_data_path = "/content/"
            base_output_path = "/content/"
            environment_name = "colab"
        elif os.environ.get("KAGGLE_KERNEL_RUN_TYPE"):
            print("✅ Environment: Kaggle")
            base_data_path = "/kaggle/input/"
            base_output_path = "/kaggle/working/"
            environment_name = "kaggle"
        else:
            print("⚠️ Environment: Local/Unknown")
            base_data_path = "./data/"
            base_output_path = "./output/"
            environment_name = "local"
    except NameError:
        print("⚠️ Non-interactive session. Using local paths.")
        base_data_path = "./data/"
        base_output_path = "./output/"
        environment_name = "local"
    os.makedirs(base_output_path, exist_ok=True)
    print(f"📂 Data Path: {base_data_path}")
    print(f"📦 Output Path: {base_output_path}")
    return base_data_path, base_output_path, environment_name
def load_secret(key_name: str) -> Optional[str]:
    """
    Loads a secret key from the appropriate environment (Colab, Kaggle, or local env vars).
    Args:
        key_name (str): The name of the secret key to load (e.g., "WANDB_API_KEY", "HF_TOKEN").
    Returns:
        Optional[str]: The secret key value if found, otherwise None.
    """
    env = ENV_NAME
    secret_value = None
    print(f"Attempting to load secret '{key_name}' from '{env}' environment...")
    try:
        if env == "colab":
            from google.colab import userdata
            secret_value = userdata.get(key_name)
        elif env == "kaggle":
            from kaggle_secrets import UserSecretsClient
            user_secrets = UserSecretsClient()
            secret_value = user_secrets.get_secret(key_name)
        else: # Local environment
            secret_value = os.getenv(key_name)
        if not secret_value:
            print(f"⚠️ Secret '{key_name}' not found in the {env} environment.")
            return None
        print(f"✅ Successfully loaded secret '{key_name}'.")
        return secret_value
    except Exception as e:
        print(f"❌ An error occurred while loading secret '{key_name}': {e}")
        return None
INPUT_PATH, OUTPUT_PATH, ENV_NAME = configure_environment_paths()

✅ Environment: Google Colab
📂 Data Path: /content/
📦 Output Path: /content/


In [3]:
!uv pip install --upgrade pip
!uv pip install -r FontDiffusion/my_requirements.txt
# 3. Install PyTorch 1.13
%cd {OUTPUT_PATH}
# Force reinstall torch 1.13 to match the model's training environment
# !uv pip uninstall torch torchvision
# !uv pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
!uv pip install torch torchvision
# 4. Install other dependencies
print("\n⬇️ Installing Dependencies (Manually fixed)...")
# Install xformers compatible with Torch 1.13
!uv pip install xformers==0.0.16 -q

# Install original dependencies
!uv pip install transformers==4.33.1 accelerate==0.23.0 diffusers==0.22.0
!uv pip install gradio==4.8.0 pyyaml pygame opencv-python info-nce-pytorch kornia
# -----------------------------------------------------------------
!uv pip install lpips scikit-image pytorch-fid
!sudo apt-get update && sudo apt-get install dos2unix
!uv pip install gdown
!uv pip install wandb
print("\n✅ Environment setup complete. You can now proceed to Block 2 (Inference).")

Using Python 3.12.12 environment at: /usr
Resolved 1 package in 16ms
Audited 1 package in 0.14ms
Using Python 3.12.12 environment at: /usr
  × No solution found when resolving dependencies:
  ╰─▶ Because you require zipp==1.0.0 and zipp==3.19.2, we can conclude that
      your requirements are unsatisfiable.
/content
Using Python 3.12.12 environment at: /usr
Audited 2 packages in 90ms

⬇️ Installing Dependencies (Manually fixed)...
  × No solution found when resolving dependencies:
  ╰─▶ Because torch==1.13.1 has no wheels with a matching Python ABI tag
      (e.g., `cp312`) and xformers==0.0.16 depends on torch==1.13.1, we can
      conclude that xformers==0.0.16 cannot be used.
      And because you require xformers==0.0.16, we can conclude that your
      requirements are unsatisfiable.

      hint: You require CPython 3.12 (`cp312`), but we only found wheels for
      `torch` (v1.13.1) with the following Python ABI tags: `cp37m`, `cp38`,
      `cp39`, `cp310`, `cp311`
Using Python 

In [4]:
import gdown
%cd {OUTPUT_PATH}
if not os.path.exists("ckpt"):
  url = "https://drive.google.com/drive/folders/12hfuZ9MQvXqcteNuz7JQ2B_mUcTr-5jZ"
  gdown.download_folder(url, quiet=True, use_cookies=False)

/content


In [5]:
# @title Unzipping all archived files
import os
import glob
from zipfile import ZipFile

zip_file_paths = glob.glob(os.path.join(INPUT_PATH, '*.zip'))

if not zip_file_paths:
    print(f'No .zip files found in {INPUT_PATH}.')
else:
    for zip_file_path in zip_file_paths:
        if os.path.exists(zip_file_path):
            print(f'Unzipping {zip_file_path}...')
            !unzip -o {zip_file_path} -d ./
            print(f'Unzipping of {zip_file_path} complete.')
        else:
            print(f'Error: The file {zip_file_path} was not found (post-glob check).')

No .zip files found in /content/.


In [6]:
# @title Checking checkpoint files (.pth)
import os
import time

CHECKPOINT_DIR = os.path.join(OUTPUT_PATH, "ckpt")
print(CHECKPOINT_DIR)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
required_files = ["unet.pth", "content_encoder.pth", "style_encoder.pth"]
while True:
    missing = [f for f in required_files if not os.path.exists(f"{CHECKPOINT_DIR}/{f}")]
    if not missing:
        print("\n✅ All weights found! You can proceed to the next step.")
        break
    else:
        print(f"Waiting for files... Missing: {missing}")
        print("Upload them to the 'ckpt' folder now.")
        time.sleep(10) # Checks every 10 seconds

/content/ckpt

✅ All weights found! You can proceed to the next step.


In [7]:
import pandas as pd
import os

def convert_csv_to_chars_txt(input_csv_path: str, output_txt_path: str, column_name: str = 'word'):
    """
    Reads a CSV file, extracts text from a specified column, and writes each character
    to a new line in a plain text file.

    Args:
        input_csv_path (str): The full path to the input CSV file.
        output_txt_path (str): The full path for the output text file.
        column_name (str): The name of the column in the CSV file containing the text.
    """
    if not os.path.exists(input_csv_path):
        print(f"Error: Input CSV file not found at '{input_csv_path}'. Please ensure the file is uploaded.")
        return

    try:
        df = pd.read_csv(input_csv_path)
    except Exception as e:
        print(f"Error reading CSV file '{input_csv_path}': {e}")
        return

    if column_name not in df.columns:
        print(f"Error: Column '{column_name}' not found in the CSV file '{input_csv_path}'.")
        return

    all_characters = []
    # Ensure the column values are treated as strings before iterating over them
    for item in df[column_name].astype(str).dropna().tolist():
        for char in item:
            all_characters.append(char)

    # Ensure output directory exists
    os.makedirs(os.path.dirname(output_txt_path), exist_ok=True)

    with open(output_txt_path, "w", encoding="utf-8") as f:
        f.write("\n".join(all_characters))
    print(f"Successfully converted '{input_csv_path}' to '{output_txt_path}', with one character per line.")

# --- Example Usage (demonstration with a dummy file) ---
# As the original file 'Ds_300_ChuNom_TuTao.csv' was not found in the previous execution,
# let's create a dummy file to demonstrate the function's usage.
print("\n--- Demonstrating function with a dummy CSV file ---")
dummy_csv_path = os.path.join(OUTPUT_PATH, "dummy_data.csv")
dummy_output_txt_path = os.path.join(OUTPUT_PATH, "dummy_chars.txt")

# Create a dummy CSV file
dummy_data = {'word': ['hello', 'world', 'python']}
pd.DataFrame(dummy_data).to_csv(dummy_csv_path, index=False)
print(f"Created a dummy CSV file at: {dummy_csv_path}")

convert_csv_to_chars_txt(dummy_csv_path, dummy_output_txt_path)

# --- How to use with your actual file ---
# Uncomment the lines below and replace 'your_actual_file.csv' and 'your_output.txt'
# with the correct paths for your use case.
#
# original_csv_file = os.path.join(INPUT_PATH, "Ds_300_ChuNom_TuTao.csv") # Or the full path to your CSV
# original_output_txt = os.path.join(OUTPUT_PATH, "nom_tu_tao.txt") # Or your desired output path
# convert_csv_to_chars_txt(original_csv_file, original_output_txt)



--- Demonstrating function with a dummy CSV file ---
Created a dummy CSV file at: /content/dummy_data.csv
Successfully converted '/content/dummy_data.csv' to '/content/dummy_chars.txt', with one character per line.


In [8]:
!ls -lart {OUTPUT_PATH}

total 44
drwxr-xr-x  4 root root 4096 Dec  9 14:41 .config
drwxr-xr-x  1 root root 4096 Dec  9 14:42 sample_data
drwxr-xr-x  1 root root 4096 Dec 30 15:18 ..
drwxr-xr-x  2 root root 4096 Dec 30 15:24 ckpt
drwxr-xr-x  9 root root 4096 Dec 30 15:42 my_dataset
drwxr-xr-x  3 root root 4096 Dec 30 15:48 outputs
drwxr-xr-x  3 root root 4096 Dec 30 15:48 wandb
drwxr-xr-x  1 root root 4096 Dec 30 15:52 .
drwxr-xr-x 12 root root 4096 Dec 30 15:52 FontDiffusion
-rw-r--r--  1 root root   24 Dec 30 15:52 dummy_data.csv
-rw-r--r--  1 root root   31 Dec 30 15:52 dummy_chars.txt


In [9]:
%cd {OUTPUT_PATH}
HF_TOKEN = load_secret("HF_TOKEN")
HF_USERNAME = "dzungpham"

# ==========================================
# EXPORT / DOWNLOAD DATASET COMMANDS
# ==========================================

# Train Split
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/train" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "train" \
  --token HF_TOKEN

!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/train_original" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "train_original" \
  --token HF_TOKEN

# Validation: Unseen Both
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/val_unseen_both" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "val_unseen_both" \
  --token HF_TOKEN

# Validation: Seen Style, Unseen Char
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/val_seen_style_unseen_char" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "val_seen_style_unseen_char" \
  --token HF_TOKEN

# Validation: Unseen Style, Seen Char
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/val_unseen_style_seen_char" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "val_unseen_style_seen_char" \
  --token HF_TOKEN

/content
Attempting to load secret 'HF_TOKEN' from 'colab' environment...
✅ Successfully loaded secret 'HF_TOKEN'.

FONTDIFFUSION DATASET EXPORTER
✓ Created output directories:
  Root: my_dataset/train
  Content: my_dataset/train/ContentImage
  Target: my_dataset/train/TargetImage

LOADING DATASET

Loading from Hub: dzungpham/font-diffusion-generated-data
README.md: 1.30kB [00:00, 5.75MB/s]
data/train-00000-of-00001.parquet: 100% 25.7M/25.7M [00:04<00:00, 5.95MB/s]
data/val_unseen_both-00000-of-00001.parq(…): 100% 307k/307k [00:01<00:00, 195kB/s]
data/val_seen_style_unseen_char-00000-of(…): 100% 940k/940k [00:02<00:00, 448kB/s]
data/val_unseen_style_seen_char-00000-of(…): 100% 6.88M/6.88M [00:02<00:00, 2.67MB/s]
data/train_original-00000-of-00001.parqu(…): 100% 68.6M/68.6M [00:05<00:00, 11.5MB/s]
data/test-00000-of-00001.parquet: 100% 85.9k/85.9k [00:01<00:00, 61.6kB/s]
Generating train split: 100% 1650/1650 [00:00<00:00, 20342.68 examples/s]
Generating val split: 100% 24/24 [00:00<00:

In [ ]:
# !uv pip install --upgrade diffusers "huggingface-hub>=0.15.1,<1.0"
# !uv pip install --upgrade accelerate peft

# already change sample_batch file to save all data in train_original
%cd {OUTPUT_PATH}
!python FontDiffusion/sample_batch.py \
    --characters "FontDiffusion/NomTuTao/Ds_10k_ChuNom_TuTao.txt" \
    --style_images "FontDiffusion/styles_images" \
    --ckpt_dir "ckpt/" \
    --ttf_path "FontDiffusion/fonts" \
    --output_dir "my_dataset/train_original" \
    --resume_from "my_dataset/train_original/results.json" \
    --num_inference_steps 20 \
    --guidance_scale 7.5 \
    --start_line 100 \
    --end_line 500 \
    --batch_size 24 \
    --save_interval 5 \
    --channels_last \
    --seed 42 \
    --compile \
    --enable_xformers

/content
/usr/local/lib/python3.12/dist-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists
pygame 2.6.1 (SDL 2.28.4, Python 3.12.12)
Hello from the pygame community. https://www.pygame.org/contribute.html

FONTDIFFUSER STANDARD FORMAT GENERATION (Index-based)
Loading characters from lines 100 to 500 (total: 10174 lines)
Successfully loaded 401 single characters.

Initializing font manager...

Loading 15 fonts from directory...
error: XDG_RUNTIME_DIR not set in the environment.
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating stri

In [15]:
!python FontDiffusion/create_validation_split.py \
  --data_root my_dataset \
  --val_ratio 0.2 \
  --test_ratio 0.1 \
  --seed 42


FONTDIFFUSION VALIDATION SPLIT CREATOR
✓ Using source directory: my_dataset/train_original

CREATING DATA SPLITS

ANALYZING TRAINING DATA

Found:
  Styles: 15 - ['style0', 'style1', 'style10', 'style11', 'style12', 'style13', 'style14', 'style2', 'style3', 'style4', 'style5', 'style6', 'style7', 'style8', 'style9']
  Characters: 701 - ['char0', 'char1', 'char10', 'char100', 'char101', 'char102', 'char103', 'char104', 'char105', 'char106']...

CREATING VALIDATION SCENARIOS

📊 Split Statistics:
  Styles: 11 train + 3 val + 1 test
  Chars:  491 train + 140 val + 70 test

📋 Validation Scenarios:

  train:
    Description: Seen styles + Seen characters (training data)
    Styles: ['style8', 'style4', 'style13', 'style10', 'style9', 'style7', 'style14', 'style3', 'style2', 'style11', 'style12']
    Chars: ['char505', 'char324', 'char318', 'char61', 'char322']...

  val_seen_style_unseen_char:
    Description: Seen styles + Unseen characters
    Styles: ['style8', 'style4', 'style13', 'style

In [16]:
# --- RAW DATA (Before Splitting) ---
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/train_original" \
  --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
  --split "train_original" \
  --private \
  --token "{HF_TOKEN}"

# --- ORGANIZED SPLITS (After Splitting) ---

# Train Split
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/train" \
  --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
  --split "train" \
  --private \
  --token "{HF_TOKEN}"

# Test Split
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/test" \
  --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
  --split "test" \
  --private \
  --token "{HF_TOKEN}"

# Validation: Unseen Both
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/val_unseen_both" \
  --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
  --split "val_unseen_both" \
  --private \
  --token "{HF_TOKEN}"

# Validation: Seen Style, Unseen Char
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/val_seen_style_unseen_char" \
  --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
  --split "val_seen_style_unseen_char" \
  --private \
  --token "{HF_TOKEN}"

# Validation: Unseen Style, Seen Char
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/val_unseen_style_seen_char" \
  --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
  --split "val_unseen_style_seen_char" \
  --private \
  --token "{HF_TOKEN}"


FONTDIFFUSION DATASET CREATOR
✓ Validated directory structure
  Content images: my_dataset/train_original/ContentImage
  Target images: my_dataset/train_original/TargetImage

BUILDING DATASET
Traceback (most recent call last):
  File "/content/FontDiffusion/create_hf_dataset.py", line 317, in <module>
    dataset = create_and_push_dataset(
              ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/FontDiffusion/create_hf_dataset.py", line 279, in create_and_push_dataset
    dataset = builder.build_dataset()
              ^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/FontDiffusion/create_hf_dataset.py", line 77, in build_dataset
    metadata = self._load_results_metadata()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/FontDiffusion/create_hf_dataset.py", line 56, in _load_results_metadata
    return json.load(f)
           ^^^^^^^^^^^^
  File "/usr/lib/python3.12/json/__init__.py", line 293, in load
    return loads(fp.read(),
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python

In [17]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()

93

In [18]:
# Training phase 1
import wandb
wandb.login(key=os.getenv("WANDB_API_KEY"))
!accelerate launch FontDiffusion/my_train.py \
    --seed=123 \
    --experience_name="FontDiffuser_training_phase_1" \
    --data_root="my_dataset" \
    --output_dir="outputs/FontDiffuser" \
    --report_to="wandb" \
    --resolution=96 \
    --style_image_size=96 \
    --content_image_size=96 \
    --content_encoder_downsample_size=3 \
    --channel_attn=True \
    --content_start_channel=64 \
    --style_start_channel=64 \
    --train_batch_size=8 \
    --perceptual_coefficient=0.03 \
    --offset_coefficient=0.7 \
    --max_train_steps=100 \
    --ckpt_interval=50 \
    --gradient_accumulation_steps=2 \
    --log_interval=50 \
    --learning_rate=1e-4 \
    --lr_scheduler="linear" \
    --lr_warmup_steps=10000 \
    --drop_prob=0.1 \
    --mixed_precision="no"

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
/usr/local/lib/python3.12/dist-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists
pygame 2.6.1 (SDL 2.28.4, Python 3.12.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
Load the down block  DownBlock2D
Load the down block  MCADownBlock2D
The style_attention cross attention dim in Down Block 1 layer is 1024

In [ ]:
# Training phase 2
!wandb login
!accelerate launch FontDiffusion/my_train.py \
    --seed=123 \
    --experience_name="FontDiffuser_training_phase_2" \
    --data_root="my_dataset" \
    --output_dir="outputs/FontDiffuser" \
    --report_to="wandb" \
    --phase_2 \
    --phase_1_ckpt_dir="outputs/FontDiffuser/global_step_2000" \
    --scr_ckpt_path="ckpt/scr_210000.pth" \
    --sc_coefficient=0.05 \
    --num_neg=13 \
    --resolution=96 \
    --style_image_size=96 \
    --content_image_size=96 \
    --content_encoder_downsample_size=3 \
    --channel_attn=True \
    --content_start_channel=64 \
    --style_start_channel=64 \
    --train_batch_size=8 \
    --perceptual_coefficient=0.03 \
    --offset_coefficient=0.4 \
    --max_train_steps=100 \
    --ckpt_interval=50 \
    --gradient_accumulation_steps=2 \
    --log_interval=50 \
    --learning_rate=1e-5 \
    --lr_scheduler="constant" \
    --lr_warmup_steps=1000 \
    --drop_prob=0.1 \
    --mixed_precision="no"


In [20]:
!python /content/FontDiffusion/pth2safetensores.py \
    --weights_dir "ckpt" \
    --repo_id "dzungpham/font-diffusion-weights" \
    --token "{HF_TOKEN}"


PYTORCH TO SAFETENSORS CONVERTER & HF UPLOADER

VALIDATING INPUTS
✓ Weights directory: ckpt
  Contents: 4 files
✓ Repository ID: dzungpham/font-diffusion-weights
✓ HF token: ********************

✓ Files to process: 5
  ✓ content_encoder.pth (4.54 MB)
  ✓ style_encoder.pth (78.59 MB)
  ✓ unet.pth (300.55 MB)
  ⚠ total_model.pth (not found)
  ⚠ scr.pth (not found)

CONVERTING .pth TO SAFETENSORS

📦 content_encoder.pth
  ✓ Saved to: ckpt/content_encoder.safetensors
    Original: 4.54 MB → Safetensors: 4.54 MB
    Compression: 0.2%

📦 style_encoder.pth
  ✓ Saved to: ckpt/style_encoder.safetensors
    Original: 78.59 MB → Safetensors: 78.58 MB
    Compression: 0.0%

📦 unet.pth
  ✓ Saved to: ckpt/unet.safetensors
    Original: 300.55 MB → Safetensors: 300.34 MB
    Compression: 0.1%

⚠ total_model.pth: Not found, skipping

⚠ scr.pth: Not found, skipping

----------------------------------------------------------------------
Conversion complete: 3 succeeded, 0 failed

UPLOADING TO HUGGING F

In [21]:
import os
import zipfile
from pathlib import Path
from typing import List
def find_result_folders(base_path: Path, pattern_name: str) -> List[Path]:
    return [p for p in base_path.glob(pattern_name) if p.is_dir()]

def zip_folder(folder_path: Path, output_base_path: Path) -> bool:
    folder_name = folder_path.name
    zip_path = output_base_path / f"{folder_name}.zip"
    try:
        print(f"   -> Zipping folder: {folder_name}...")
        with zipfile.ZipFile(zip_path, mode="w", compression=zipfile.ZIP_DEFLATED) as zipf:
            for file_path in folder_path.rglob("*"):
                if file_path.is_file():
                    arcname = file_path.relative_to(folder_path.parent)
                    zipf.write(file_path, arcname)
        print(f"   ✅ Created ZIP: {zip_path.name}")
        return True
    except Exception as exc:
        print(f"   ❌ Failed to zip {folder_name}: {exc}")
        return False

def zip_stats_results_folders(output_base_path: str, pattern_name: str) -> None:
    base = Path(output_base_path)
    base.mkdir(parents=True, exist_ok=True)
    result_folders = find_result_folders(base, pattern_name)
    if not result_folders:
        print(f"⚠️ No folders matching '*dataset' found in '{output_base_path}'.")
        return
    print(f"🔍 Found {len(result_folders)} result folder(s) to zip.")
    successful = sum(1 for folder in result_folders if zip_folder(folder, base))
    print(f"\n✅ DONE! Successfully zipped {successful} out of {len(result_folders)} folder(s).")

if __name__ == "__main__":
    try:
        output_root = os.getenv("OUTPUT_PATH") or globals().get("OUTPUT_PATH")
        if not output_root:
            raise ValueError("OUTPUT_PATH not defined")
        zip_stats_results_folders(
            output_base_path=OUTPUT_PATH,
            pattern_name="outputs/FontDiffuser/global_step_50")
    except Exception as e:
        print(f"❌ An error occurred: {e}")

⚠️ No folders matching '*dataset' found in '/content/'.
